In [2]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

import graphviz 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.metrics import top_k_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score

from frouros.detectors.concept_drift import DDM, DDMConfig
from frouros.metrics import PrequentialError

### Carregamento do conjunto de dados de todos os anos

In [3]:
enade_todos_anos = pd.read_csv("../../teste_OHE_curso_treinamento.csv")

# enade_todos_anos = pd.read_csv("../../tabela_BCC.csv")

### Cross-validation (k = 5)

In [4]:
k = 5

In [5]:
array_folds = np.array_split(enade_todos_anos, k)

In [6]:
# Detector configuration and instantiation
config = DDMConfig(
    warning_level=2.0,
    drift_level=3.0,
    min_num_instances=25,  # minimum number of instances before checking for concept drift
)
detector = DDM(config=config)

In [7]:
# Metric to compute accuracy
metric = PrequentialError(alpha=1.0)  # alpha=1.0 is equivalent to normal accuracy

In [8]:
drift_flag = False

In [ ]:
for i_fold in range(k):
    
    # Separação dos folds
    
    folds_treinamento = array_folds.copy()
    fold_teste = array_folds[i_fold]
    
    del folds_treinamento[i_fold]
    
    folds_treinamento = pd.concat(folds_treinamento, sort=False)
    
    # Separação em X e y
    
    numero_caracteristicas = enade_todos_anos.shape[1] - 1
    
    X_folds_treinamento = folds_treinamento.iloc[:, 0:numero_caracteristicas]
    y_folds_treinamento = folds_treinamento.iloc[:, -1]
    
    X_fold_teste = fold_teste.iloc[:, 0:numero_caracteristicas]
    y_fold_teste = fold_teste.iloc[:, -1]
    
    # Normalização de treino e teste
    
    normalizador_treinamento = StandardScaler()
    
    normalizador_treinamento.fit(X_folds_treinamento)
    treinamento_normalizado = normalizador_treinamento.transform(X_folds_treinamento)
    
    normalizador_teste = StandardScaler()
    
    normalizador_teste.fit(X_fold_teste)
    teste_normalizado = normalizador_teste.transform(X_fold_teste)
    
    # Busca por hyperparâmetros
    
    parametros_para_busca = {"C": [0.01, 0.1, 1, 10, 100], 
              "kernel": ["linear", "poly", "rbf"],
              "gamma": [0.1, 1, 10],
              "tol": [0.1, 0.001, 0.00001]
             }
    
    classificador = SVC(probability=True)
    
    busca = HalvingGridSearchCV(classificador, parametros_para_busca, scoring="f1_weighted").fit(treinamento_normalizado, y_folds_treinamento)
    
    busca.best_estimator_.fit(treinamento_normalizado, y_folds_treinamento)
    y_predito = busca.best_estimator_.predict(teste_normalizado)
    
    error = 1 - (y_predito.tolist() == y_fold_teste.tolist())
    metric_error = metric(error_value=error)
    _ = detector.update(value=error)
    status = detector.status
    
    if status["drift"] and not drift_flag:
        drift_flag = True
        print(f"Concept drift detected at step {i}. Accuracy: {1 - metric_error:.4f}")
    
    print(f"Final accuracy: {1 - metric_error:.4f}\n")
    
    melhor_classificador = str(busca.best_estimator_)
    resultado = f1_score(y_fold_teste, y_predito, average='weighted')
    
    #print(f"Teste: fold {i_fold}")
    #print(f"Classificador: \n\n{melhor_classificador}\n")
    print(f"Resultado: {resultado}\n")
    
if not drift_flag:
    print("No concept drift detected")

### Matriz de Confusão

In [ ]:
y_pred_HGS = search.predict(X_test)

In [ ]:
cm_HGS = confusion_matrix(y_test, y_pred_HGS, labels=search.classes_)
disp_HGS = ConfusionMatrixDisplay(confusion_matrix=cm_HGS,
                              display_labels=search.classes_)
disp_HGS.plot()
plt.show()